# 尝试使用 fasttext 进行文本分类

In [1]:
import jieba
import pandas as pd
df_pos = pd.read_csv('./data/pos.csv', encoding = 'utf-8', sep='\t')
df_pos = df_pos.dropna()

df_neg = pd.read_csv('./data/neg.csv', encoding = 'utf-8', sep='\t')
df_neg = df_neg.dropna()


pos = df_pos.content.values.tolist()[1000:51000]
neg = df_neg.content.values.tolist()[1000:51000]

In [4]:
stopwords = pd.read_csv('./rowdata/stopwords.txt', 
                        index_col=False, quoting=3, sep='\t', names=['stopword'], encoding='utf-8')
stopwords = stopwords['stopword'].values

In [5]:
def preprocess_text(content_lines, sentences, category):
    for line in content_lines:
        try:
            segs = jieba.lcut(line)
            segs = filter(lambda x:len(x)>1, segs)
            segs = filter(lambda x:x not in stopwords, segs)
            sentences.append((" ".join(segs), category))
        except Exception:
            continue
sentences = []

preprocess_text(pos, sentences, 'pos')
preprocess_text(neg, sentences, 'neg')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/tb/k5xqd901235gk634ymvtk5140000gn/T/jieba.cache
Loading model cost 0.937 seconds.
Prefix dict has been built successfully.


In [6]:
import random
random.shuffle(sentences)

In [7]:
from sklearn.model_selection import train_test_split
x, y = zip(* sentences)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 1234)

In [8]:
len(x_train)

75000

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(
    analyzer='word', 
    max_features=4000
)
vec.fit(x_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=4000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [20]:
def get_features(x):
    vec.transform(x)

In [21]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(vec.transform(x_train), y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [22]:
classifier.score(vec.transform(x_test), y_test)

0.9

In [23]:
len(x_test)

25000

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(
    analyzer='word', # tokenise by character ngrams
    ngram_range=(1,4),  # use ngrams of size 1 and 2
    max_features=20000,  # keep the most common 1000 ngrams
)
vec.fit(x_train)

def get_features(x):
    vec.transform(x)

In [26]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(vec.transform(x_train), y_train)
classifier.score(vec.transform(x_test), y_test)

0.90356

In [27]:
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB


class TextClassifier():

    def __init__(self, classifier=MultinomialNB()):
        self.classifier = classifier
        self.vectorizer = CountVectorizer(analyzer='word', ngram_range=(1,4), max_features=20000)

    def features(self, X):
        return self.vectorizer.transform(X)

    def fit(self, X, y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.features(X), y)

    def predict(self, x):
        return self.classifier.predict(self.features([x]))

    def score(self, X, y):
        return self.classifier.score(self.features(X), y)

In [29]:
text_classifier = TextClassifier()
text_classifier.fit(x_train, y_train)
print(text_classifier.predict('这是有史以来最垃圾的公司'))
print(text_classifier.score(x_test, y_test)) # 准确度

['neg']
0.90356


In [34]:
text_classifier.predict('支持 维权，我看好你！加油')[0]

'neg'

In [35]:
def comPreProcess(text):
    sentence = []
    segs = jieba.lcut(text)
    segs = filter(lambda x: len(x)>1, segs)
    segs = filter(lambda x: x not in stopwords, segs)
    sentence.append(' '.join(segs))
    return sentence[0]

comPreProcess("你是个大笨蛋，我讨厌你，你可太过分了啊")

'笨蛋 讨厌 太过分'

In [37]:
text = '你是个大笨蛋，我讨厌你，你可太过分了啊'
text_classifier.predict(comPreProcess(text))

array(['neg'], dtype='<U3')

In [ ]:
df_pl_content = pd.read_csv('./data/BiliComment_contain20000.csv', sep='\t')

df_pl_content = df_pl_content.content.values.tolist()

for i in df_pl_content:
    print(i,text_classifier.predict(comPreProcess(i)))  # 除了分词还可以过滤下 停用词，写个函数啥的

也是上过央视哈哈哈哈哈哈 ['pos']
恶意抢注 ['neg']
加油 ['pos']
老爹说过要用魔法对抗魔法        出自《成龙历险记》。 ['pos']
支持维权 ['neg']
身份证不见了还可以去改名吗？急，打算明天去公安局改的，结果身份证不见了，真的想哭，已经成年了，只带户口本可以去改吗？ ['neg']
加油 ['pos']
真他妈的可悲 ['neg']
2507.4w ['neg']
一杯敬死神，一杯敬汉卿。 ['neg']
加油 ['pos']
加油 ['pos']
皮皮虾和b站都关注了 ['pos']
加油⛽️ ['pos']
冲冲冲 ['neg']
这事本身就很荒唐，人家名字也可以注册？呵呵笑死我了，这个部门在干嘛 ['neg']
真的没想到敬汉卿是真名，父母太有才了，取的好好听 ['pos']
第一次三连[OK] ['pos']
太难了up主 ['neg']
一直喜欢敬汉卿大大的视频，支持依法维权，坚决抵制这种不道德的行为。 ['neg']
支持你 ['pos']
敬汉卿你好，我也经常看你的节目，我们也是做商标这块的。刚才查询了一下，目前这个41还不是最要命的，你的35类也被抢注了，现在正在公示，如果35类被抢注，你就不能做广告，彻底凉凉了，希望你能引起共重视，现在赶紧去。35类的商标进行复议 复制自热心网友35类已经在2019.7.13号开始初审公示了，如果在三个月之内，也就是10月13号之前没有人复议的话就通过了，你35类商标如果丢了的话，你彻底就告别这个名字了（复制粘贴评论区）敬汉卿你好，我也经常看你的节目，我们也是做商标这块的。刚才查询了一下，目前这个41还不是最要命的，你的35类也被抢注了，现在正在公示，如果35类被抢注，你就不能做广告，彻底凉凉了，希望你能引起共重视，现在赶紧去。35类的商标进行复议 复制自热心网友35类已经在2019.7.13号开始初审公示了，如果在三个月之内，也就是10月13号之前没有人复议的话就通过了，你35类商标如果丢了的话，你彻底就告别这个名字了（复制粘贴评论区） ['neg']
抵制非法经营，保护维护自名正当权益，保护姓名并提供法律法规政策支持。敬汉卿，加油！[打call] ['neg']
奥利给 ['neg']
加油[微笑] ['pos']
就服你_x1f44d__x1f44d__x1f44d

加油！ ['pos']
加油，奥利给！！！ ['pos']
弄他们！！！那些讹钱的垃圾！！！！弄他们！！！！ ['neg']
你很出名！！！！！！！你很出名！！！！！！！你很出名！！！！！！！！！ ['neg']
我子弟斯 ['neg']
加油 ['pos']
别说了 别说了 我祝你一臂之力 ['neg']
加油鸭 ['pos']
卧槽，要这么多证明。。。。我当初为何不去学法律专业？我陷入了沉思。。。[捂脸] ['neg']
来的有点晚了，加油！ ['pos']
加油 ['pos']
加油 ['pos']
加油 奥力给 ['pos']
Up不用太担心，即使官司输了，这个公司也能倒赔几百万，他使用了微软雅黑字体，这个字体用来当商标是侵权的 ['pos']
哈哈哈 ['pos']
确定是22年？32年吧不要四舍五入哦[doge][滑稽] ['neg']
up主很有名，支持维权 ['neg']
你很出名[打call] ['pos']
支持UP ['pos']
加油，你是最棒的 ['pos']
恭喜你找到了[热词系列_镇站之宝] ['neg']
加油 ['pos']
感觉在斗地主[抠鼻] ['neg']
a ['neg']
支持！！ ['pos']
我觉得up的名字好好听完哇⊙∀⊙！ ['neg']
支持维权！ ['neg']
加油加油加油 ['pos']
加油↖(^ω^)↗ ['pos']
自己全类注册一下 ['neg']
第一个那个评论是什么鬼……我这边怎么有他黑名单。 ['neg']
好的！ ['neg']
今天再看播放量竟高达2400万＋[打call] ['neg']
OK很出名 ['pos']
加油加油 ['pos']
支持维权 ['neg']
不光赞，还投币 ['neg']
支持！ ['pos']
暗示加大力度 ['neg']
@席拉瑞思 来帮帮忙呗[洛天依_去吧] ['neg']
加油 ['pos']
加油！！ ['pos']
一nnnnnn ['neg']
加油 ['pos']
dd ['neg']
[doge] ['neg']
(⌒▽⌒)(⌒▽⌒) ['neg']
敬请放心 ['pos']
支持支持[打call] ['pos']
这名字不错 ['pos']
你真的很出名！！！ ['neg']
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出

建议国家出台政策严惩这些钻法律漏洞的人，国家法律威严！不容侵犯！现在越来越多这样的事情发生，就说明越来越多的人都在做这样的事情，一个国家的法律摆在这里，是让一些人用来天天想着怎么搞一个漏洞得点好处的吗？为什么这种事情会频发，这是法律啊，时间长了威严何在？ ['neg']
指法芬芳张大仙被斗鱼作为商标注册了→_→[阴险] ['neg']
加油＾０＾~ ['pos']
支持！ ['pos']
不要跟那种垃圾公司客气 ['neg']
up加油！ ['pos']
顶 ['neg']
支持 ['pos']
冲 ['neg']
为什么这种垃圾公司还好意思活着 ['neg']
敬汉卿粉丝700万了。 ['neg']
加油 ['pos']
不过是实话 ['neg']
加油 ['pos']
支持_x1f601_ ['pos']
加油！ ['pos']
支持合法维权！ ['neg']
以后给孩子起名字需要先去注册一下了，嗯名字就叫骆99999999999999999 ['neg']
打）气！ ['neg']
知名！！！！！ ['neg']
默默支持你 ['pos']
加油! ['pos']
打气 ['neg']
真名估计是那帮人没想到的 ['neg']
你很出名 ['neg']
up猪加油啊 ['pos']
加油_x1f4aa_ ['pos']
加油 ['pos']
来吧 ['neg']
这种公司确实作死 ['pos']
加油加油 ['pos']
支持维权 ['neg']
加油_x1f4aa_ ['pos']
阿婆主加油 ['pos']
卧槽.这条视频把大佬全炸出来了 ['neg']
看着发的邮件_x1f4e7_读起来太没文化了……支持 ['pos']
评论一下 ['neg']
[惊喜][惊喜][惊喜][惊喜] ['pos']
你们注册商标，和我取的名字有什么关系 ['neg']
加油 ['pos']
我真的不认识你[喜极而泣] ['neg']
太可怕了……………[大哭] ['neg']
加油୧(๑•̀⌄•́๑)૭膝盖正义 ['pos']
笑死我了，心疼你 ['neg']
加油 ['pos']
加油支持 ['pos']
我家用了10多年的店名前年才注册，刚注册就有人抢注还不停有人打电话骚扰，还有以各种关于商标诈骗电话，真搞不懂这些信息电话是怎么到他们手里了 ['neg']
666666

你很出名 ['neg']
不认可这是违法，使用自己的名字没有错 ['neg']
已经三连，加油 ['pos']
加油 ['pos']
走你 ['neg']
加油！ ['pos']
干吧哟，OK，打气加油了！[吃瓜] ['pos']
我有一个朋友是西南政法大学硕士在读的，专攻知识产权方面的，不知道对你有没有帮助[呆] ['pos']
加油 ['pos']
支持你 ['pos']
顶 ['neg']
所以事情后续到底怎样了 ['neg']
支持！！！ ['pos']
好的 ['neg']
三连 ['neg']
Up加油 ['pos']
加油 ['pos']
我甚至想给你点个关注 ['pos']
可怜啊 ['neg']
加油！！！？ ['pos']
我看这个视频 认识到了商标 然后我即将要开始学商标法了 ['neg']
嗯嗯嗯嗯 ['neg']
支持维权 ['neg']
这是最多硬币的作品了。 ['pos']
加油！你火了！ ['pos']
加油，顶你 ['pos']
好的朋友 ['pos']
加油加油 ['pos']
嘎嘎嘎嘎 ['neg']
疯狂暗示，请问那家公司叫什么名 ['neg']
(⁄ ⁄•⁄ω⁄•⁄ ⁄) ['neg']
不对啊，名字和商标能有关系，那个叫支付宝的大爷。。。 ['neg']
支持up主 ['pos']
支持支持 ['pos']
支持！ ['pos']
每天评论 ['neg']
支持合法维权 ['neg']
666 ['pos']
pxj也报告，请指示 ['neg']
好了 让你上去! ['neg']
加油＾０＾~ ['pos']
他们万万没有想到，我的真名就叫德克萨斯[滑稽] ['neg']
支持 ['pos']
加油。 ['pos']
我不是很懂，自己的名字为什么要注册？不出名的不配拥有名字吗？ ['neg']
1111 ['neg']
支持你。 ['pos']
支持敬汉卿 ['pos']
又上我的推荐了 ['pos']
。。。 ['neg']
评论了 顶你 ['neg']
我觉得注册名字作为商标应该不能被承认，按照这种说法，我注册先辈伟人，两位马总的名字，岂不是他们也要给钱。这一看就知道是不成立的，更何况全球同名同姓的人这么多，注册了明显出问题。面对这种事情必须维权，要不然也不知道坑害多少人。 ['neg']
名字是父母给的，就算是世界政

2300.0w播放量 ['neg']
66关注 666.6万粉丝[微笑] ['pos']
请法律顾问了吗 ['neg']
支持你，加油 ['pos']
卧槽四百多万硬币 ['neg']
冲冲冲 ['neg']
j ['neg']
666.6 ['pos']
666666 ['neg']
支持维权！ ['neg']
敬汉卿nbnbnb ['neg']
小心着点吧，我盯上你_x1f643_ ['neg']
支持up主 ['pos']
666.6万粉丝666 ['pos']
厉害了 ['neg']
2290合影 ['neg']
没有，你好惨呀！[酸了][酸了][酸了][酸了][酸了][酸了][酸了][酸了][酸了][酸了][酸了][酸了] ['neg']
等你 ['neg']
评论全是 名 人 堂 ['neg']
加油！ ['pos']
支持兄弟合法权益 ['neg']
史上最惨镇站之宝[蛆音娘_滑稽] ['neg']
加油 ['pos']
iiiii ['neg']
史上最惨镇站之宝 ['neg']
三连，加油！ ['pos']
这些碰瓷式的专利和版权公司是真的恶心 ['neg']
我想知道为什么这么一个普普通通的视频，为什么会有这么高的播放量，刷的吧，部分 ['neg']
我………我是怎么在搞笑区看到这个视频的？ ['neg']
加油 ['pos']
支持~ ['pos']
一杯敬死神，一杯敬汉卿 ['neg']
。。 ['neg']
加油 ['pos']
支持维权 ['neg']
支持维权 ['neg']
这就是名人的苦恼 我们这种咸鱼也是不会担心这种事的 咸鱼使我开心 ['pos']
11 ['neg']
这年头，我反而还得想办法证明我为什么是我。。真的太难了。。 ['neg']
加油 ['pos']
评论这么多，可怕呀。 ['neg']
支持汉卿！ ['pos']
盖楼， ['neg']
好的好的，举手之劳帮帮你。 ['neg']
up加油 ['pos']
最后怎么样了现在 ['neg']
[酸了][大哭] ['neg']
加油up主 ['pos']
[滑稽] ['neg']接白嫖oge]
支持维权！ ['neg']
把公司IP黑一下就完事，熊猫烧香预备 ['neg']
_x1f62d__x1f62d_我也被盗名了，我是个作家（笔名不是这个） ['neg']
我

...

加油加油 ['pos']
敬汉卿加油，那帮sb的所作所为不会击垮你，他们会让你更火滴 ['pos']
加油！ ['pos']
加油加油，希望打赢 ['pos']
加油！我们会支持你的！ ['pos']
上人民日报了 ['neg']
[无语]这种事真的很多，真是很恼火 ['neg']
支持你 ['pos']
支持！ ['pos']
支持 ['pos']
加油_x1f4aa_ ['pos']
其实律师这种事找cxk就OK了 ['pos']
加油 ['pos']
支持 ['pos']
加油啊！！ ['pos']
加油敬汉卿 ['pos']
支持！！！ ['pos']
支持！加油！ ['pos']
哇哦⊙∀⊙！ ['neg']
加油！ ['pos']
加油 ['pos']
加油，一定可以维权成功的 ['neg']
加油加油 ['pos']
加油挺你 ['pos']
emmm，知识产权局局长是我们大学校友。。。 ['pos']
提问，假设我的名字也被人注册了，但我毫不知情。某一天我用我的名字当ID在小破站当UP主上传视频，然后慢慢的火起来了，这个时候商标拥有人说我侵权律师函警告改名字，若是我不改和他对簿公堂的话有没有赢得可能？ ['neg']
支持维权，用法律维护自身权益 ['neg']
中国法制国家会给大家一个解释的 ['neg']
加油 ['pos']
加油加油 ['pos']
Up加油！ ['pos']
此条评论真实有效，且只发一次，我愿意承担我所说的一切法律责任。 ['neg']
能不能曝光这家公司 ['neg']
支持合法维权！ ['neg']
好多大佬都在啊[微笑][微笑] ['neg']
up主加油！你的节目我很喜欢_x1f618_一定要大赢！ ['pos']
支持维权 ['neg']
支持UP主 ['pos']
加油[微笑] ['pos']
我也只能点个赞投个币转发一下了[doge] ['neg']
最看不惯这明白就是欺负人的事，想知道后续，别给他们钱！ ['neg']
四天一千四百万，恐怖[惊讶][惊讶] ['neg']
1500万，冲鸭 ['neg']
《动不动说分手》 ['neg']
加油 ['pos']
支持 ['pos']
我就不相信有人注册我这个名字 ['neg']
姓名是姓名。商标是商标。 ['neg']
敬汉卿加油 ['pos']
被平台老板玩

打卡 ['neg']
加油！！ ['pos']
加油啊 ['pos']
我来证明敬汉卿真的很有名气 ['neg']
加油敬汉卿！！！ ['pos']
这是我在b站看过的播放量最多的视频 ['neg']
维权到底加油！！！ ['neg']
为了支持你，我三连了，希望以后出后续发展情况的汇报 ['pos']
小伙伴们挺起来 ['neg']
用魔法打败魔法 ['neg']
加油呀 ['pos']
加油 ['pos']
一点点加油给你，加油！ ['pos']
加油 ['pos']
加油↖(^ω^)↗ ['pos']
可怕的播放量。。。 ['neg']
加油 ['pos']
维权！！！ ['neg']
保护我方up主[蛆音娘_生气][蛆音娘_生气][蛆音娘_生气] ['neg']
支持支持支持 ['pos']
加油！！ ['pos']
加油！！！！！！！ ['pos']
冲啊冲啊 ['neg']
加油 ['pos']
评个论 ['neg']
直播小心刀片 ['neg']
敬汉卿，加油_x1f4aa_维护自己的权利 ['neg']
牛逼熬 ['neg']
加油! ['pos']
希望up主能够维权成功！加油！ ['neg']
一定要加油鸭! ['pos']
支持！！ ['pos']
把我顶上去 ['neg']
流弊，支持你 ['pos']
支持合法维权！ ['neg']
支持 ['pos']
证明一哈哈233 ['neg']
这种抢注的垃圾丢到垃圾桶里垃圾桶都要恶心到吐出来[囧][囧] ['neg']
我去，这视频播放长得真得快 ['neg']
这么几个够了没 ['neg']
支持合法维权 ['neg']
今天中午的人民日报有敬汉卿这一期诶 ['pos']
加油(ง •̀_•́)ง ['pos']
加油(ง •̀_•́)ง ['pos']
支持up主，加油 ['pos']
666嗷 ['pos']
会的 ['neg']
来了来了 ['neg']
加油！刚他们！ ['pos']
我有一个疑问，为什么他西瓜视频的粉丝有600W+，每个视频播放量几万到几十万，而b站粉丝400W+，视频播放量却有几百万到一千万多万呢？ ['neg']
一杯敬死神，一杯敬汉卿 ['neg']
每天都来关注一下这个视频的热度 ['pos']
@渗透之C菌 ['neg']
支持，加油UP主 ['pos']


是我最喜欢的UP主之一！！！！ ['neg']LI各路人马检阅的。。。
支持！！！！！！ ['pos']
加油，冲冲冲 ['pos']
支持敬汉卿，关注了。加油小敬！ ['pos']
额，我一开始就认为敬汉卿是真名了。。。[吃瓜] ['neg']
评论评论！！死神之子skr ['neg']
加油 ['pos']
支持维权 ['neg']
支持[支持] ['pos']
满屏的六级大佬[尴尬][尴尬] ['neg']
加油 ['pos']
人心险恶啊，支持up主 ['neg']
冲冲冲 加油 ['pos']
加油 ['pos']
必须硬刚，挺！！！ ['neg']
支持合法维权！！！！ ['neg']
加油！[2233娘_第一] ['pos']
顶上去 ['neg']
石家庄唐牌科技有限公司，你们自己看着办 ['neg']
支持下～你很出名看你视频很久了 ['pos']
你的真名好酷！ ['neg']
上微信端的人民日报公众号了(=・ω・=)虽然转的是共青团中央 ['neg']
特地证明(￣3￣) ['neg']
加油 ['pos']
敬汉卿必赢！！！ ['neg']
支持_x1f4aa_ ['pos']
支持维权 ['neg']=・ω・=)
支持敬汉卿 ['pos']
顶顶顶 ['neg']
阿婆主加油(ง •̀_•́)ง ['pos']
支持！加油！ ['pos']
冲鸭冲鸭 ['neg']
加油 ['pos']
冲冲冲！ ['neg']
支持合法维权 ['neg']
支持！！ ['pos']
预祝500w粉丝。 ['neg']
加油！[支持][支持][支持][支持]作为成都人支持你！ ['pos']
6666666 ['neg']
现在疯狂改编[呆] ['neg']
经营范围	电子产品批发兼零售（依法须经批准的项目，经相关部门批准后方可开展经营活动 ['neg']
路人支持 ['pos']
不是说不能采取网络暴力吗？那我们就去他们家门口组团(1/1000)[滑稽][滑稽][滑稽] ['neg']
支持 ['pos']
共青团中央都关注了这件事 ['pos']
支持！ ['pos']
加油(ง •̀_•́)ง ['pos']
大家快来看讽刺此公司的鬼畜：穷抢注av62496561 ['pos']
加油，相信你。 ['pos']
这个视频算不算镇站之宝了啊[微笑] [

加油加油加油 ['pos']
支持支持！！！！[画风突变][画风突变] ['pos']
加油啊 ['pos']
支持敬汉卿！！！ ['pos']
从其他up的视频排行找到的。。。原来是这个意思 ['neg']
加油(｀・ω・´) ['pos']
什么时候能普及版权法呢？ ['neg']
加油 ['pos']
他又想怎样？ ['neg']算他真的用了你的名字，拿不出来作品，他又能怎样？
来证明 ['neg']
一定要维权！！ ['neg']
你很出名 ['neg']
AV62466841 ['neg']
太狗了 ['neg']
NB ['neg']
你很有名！！！ ['neg']
维权加油！ ['neg']
死神之子加油 ['pos']
放心~公道和正义不会偏袒这些不要脸的公司的~这种法律也会站在UP这边的~不然那些常见名或者常见网名都不能用了?不可能的~那个公司是知法犯法只知其一不知其二~先有先注册的不会被后面恶意抢注的欺负的 ['neg']
这些事情要是在贴吧知乎微博根本不叫问题 因为明眼人很多 懂法律的也很多 不过在b站 可以当一个法律科普来看 毕竟b站低龄化严重 什么都不懂的小孩子占大多数 敬汉卿真的能捞热度 正愁没素材呢 做了这个视频也不讲清楚解决办法 还说粉丝别去骚扰商家 我寻思你不想让别人去骚扰商家那你就别做这个视频了呗 要不你就直接讲讲遇到这事情 应该怎么处理不就完了 之前还一直强调自己的名字被注册了 跟自己22年的名字不能用了 这商标和你的名字有个屁关系？这种处理事的方法真的是邪门歪道 就好像你去健身房 一直用一个健身器材 姿势都调好了 然后用了22年 今天去晚了突然发现机器被占了 你就说这东西跟了我22年怎么怎么的 你咋不买下来？ ['neg']
拿起法律武器反抗 ['neg']
你很火歪[微笑] ['neg']
必须维权。 ['neg']
爱你(ɔˆ ³(ˆ⌣ˆc) ['neg']
加油 ['pos']
不行了再挺一波罗汉，毕竟这次up这个视频真的太重要，无论是对小破站up还是我们这些粉来说，真的是成功得唤醒了一些内心深处的东西，现在就到了我们这些良知人士行动的时候了！！！加油啊，各位！！！ ['neg']
支援 ['neg']
上人民日报了！ ['neg']
加油，敬汉卿！ ['pos']
加油 ['pos']
安徽人这里给大家道歉了 ['ne

啊！ ['neg']
大家快去看看av62496104 ['neg']
。。。。 ['neg']
加油 ['pos']
加油(ง •̀_•́)ง ['pos']
加油啊！！！ ['pos']
加油啊 很早就关注你了 很喜欢你 ['pos']
加油(ง •̀_•́)ง ['pos']
支持维权(=・ω・=) ['neg']
加油！ ['pos']
支持[微笑] ['neg']
虽然不是你的粉，但是还是支持你的合法维权！加油！ ['neg']
咕咕咕 ['neg']
支持 ['pos']
加油！ ['pos']
（二）在订立合同时显失公平的。 ['neg']
额，不能投币了 ['neg']
加油加油加油 ['pos']
加油＾０＾~ ['pos']
下载B站以来没评论过，真的特别喜欢敬汉卿，也想尽一点点微薄之力，死神之子一定要加油鸭！！！ ['pos']
你们说的这个抢注商标，他无耻吗？ ['neg']
如果b站实行实名制政策，这样的事就能减少很多，而且现在最致命的是身份信息泄露，建议各大实名制软件管好身份信息，严防泄露，建议加强法律管理，最好是下次再犯能判刑，这已经是集敲诈，恐吓，勒索，诽谤，骚扰等违法活动于一身的公司了叭 ['neg']
加油，敬汉卿[支持] ['pos']
支持 ['pos']
一键三连 ['pos']
更加出名了 ['neg']
支持你维权 ['neg']
镇战之宝留影 ['neg']
加油 ['pos']
支持阿婆主 ['pos']
加油加油[支持][支持][支持] ['pos']
加油啊 ['pos']
支持！ ['pos']
老高的商标也是被抢注了   之前频道都改名了 ['neg']
敬这个字做姓的时候不是读gou吗 ['neg']
捞你 ['neg']
加油 ['pos']
加油 ['pos']
我的天这个播放量………………… ['neg']
冲啊 ['neg']
加油( ´_ゝ｀) ['pos']
加油 ['pos']
喜欢喜欢 ['pos']
加油 ['pos']
加油 ['pos']
加油！ ['pos']
蔡徐坤，这个人已经成牌子了 ['pos']
加油加油 ['pos']
支持维权！ ['neg']
敬汉卿加油 ['pos']
唉，这都什么人。。。 ['neg']
加油啊 ['pos']
666 ['pos']
加油[微笑] 